In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
import os 
os.chdir("/content/drive/Hamming")

In [5]:
!pwd

/content/drive/Hamming


In [6]:
!ls

g.txt		     hamming.txt  loss.py    src.py
hamming-codes.ipynb  h.txt	  README.md  visualisation.ipynb


In [32]:
!git pull

remote: Counting objects: 6, done.
remote: Total 6 (delta 2), reused 2 (delta 2), pack-reused 4
Unpacking objects: 100% (6/6), done.
From https://github.com/aae2609/Hamming
   6d555bb..a6fd94c  master     -> origin/master
Updating 6d555bb..a6fd94c
Fast-forward
 .gitignore | 2 ++
 loss.py    | 7 +++++++
 2 files changed, 9 insertions(+)
 create mode 100644 .gitignore
 create mode 100644 loss.py


In [0]:
COLUMN_NAMES = ('id', 'plainword', 'codeword', 
                'id_error', 'bin_error', 'defective_codeword')
TRAIN_PATH = "/content/drive/Hamming/hamming.txt"

In [0]:
import pandas as pd
from sklearn import model_selection

In [0]:
data = pd.read_csv(TRAIN_PATH, sep=';', names = COLUMN_NAMES)

In [20]:
data.head()

,id,plainword,codeword,id_error,bin_error,defective_codeword,bin_0,bin_1,bin_2,bin_3,...,bin_21,bin_22,bin_23,bin_24,bin_25,bin_26,bin_27,bin_28,bin_29,bin_30
0,0,01101110111110010000111110,0100101101110111110010000111110,9,0000000001000000000000000000000,0100101100110111110010000111110,0100101100110111110010000111110,0001100110010000100100100001001,1011010011110110010011011100100,1100010001111100101010001110101,...,1110000110010001011011111001000,0001000100011000110110000011101,0110111000110110101111010011110,1011101110110001001000001001011,1101010000010110011110001100011,0101000000001111010100000010100,0111101001100111000111111011101,0101110100011101011100110001111,0011011110010010110111000110001,0100111101110111110000011101001
1,1,00110010000100100100011001,0001100110010000100100100011001,26,0000000000000000000000000010000,0001100110010000100100100001001,0100101100110111110010000111110,0001100110010000100100100001001,1011010011110110010011011100100,1100010001111100101010001110101,...,1110000110010001011011111001000,0001000100011000110110000011101,0110111000110110101111010011110,1011101110110001001000001001011,1101010000010110011110001100011,0101000000001111010100000010100,0111101001100111000111111011101,0101110100011101011100110001111,0011011110010010110111000110001,0100111101110111110000011101001
2,2,10011010110010011011100100,1011010011010110010011011100100,10,0000000000100000000000000000000,1011010011110110010011011100100,0100101100110111110010000111110,0001100110010000100100100001001,1011010011110110010011011100100,1100010001111100101010001110101,...,1110000110010001011011111001000,0001000100011000110110000011101,0110111000110110101111010011110,1011101110110001001000001001011,1101010000010110011110001100011,0101000000001111010100000010100,0111101001100111000111111011101,0101110100011101011100110001111,0011011110010010110111000110001,0100111101110111110000011101001
3,3,10001111100101011001110101,1100010001111100101011001110101,21,0000000000000000000001000000000,1100010001111100101010001110101,0100101100110111110010000111110,0001100110010000100100100001001,1011010011110110010011011100100,1100010001111100101010001110101,...,1110000110010001011011111001000,0001000100011000110110000011101,0110111000110110101111010011110,1011101110110001001000001001011,1101010000010110011110001100011,0101000000001111010100000010100,0111101001100111000111111011101,0101110100011101011100110001111,0011011110010010110111000110001,0100111101110111110000011101001
4,4,01101011110100100000001110,1010001101011110100100000001110,8,0000000010000000000000000000000,1010001111011110100100000001110,0100101100110111110010000111110,0001100110010000100100100001001,1011010011110110010011011100100,1100010001111100101010001110101,...,1110000110010001011011111001000,0001000100011000110110000011101,0110111000110110101111010011110,1011101110110001001000001001011,1101010000010110011110001100011,0101000000001111010100000010100,0111101001100111000111111011101,0101110100011101011100110001111,0011011110010010110111000110001,0100111101110111110000011101001


In [0]:
for j in range(len(data['defective_codeword'][0])):
  data['bin_' + str(j)] = data['defective_codeword'][:].apply(lambda x: int(x[j]))

In [35]:
data.head()

,id,plainword,codeword,id_error,bin_error,defective_codeword,bin_0,bin_1,bin_2,bin_3,...,bin_21,bin_22,bin_23,bin_24,bin_25,bin_26,bin_27,bin_28,bin_29,bin_30
0,0,01101110111110010000111110,0100101101110111110010000111110,9,0000000001000000000000000000000,0100101100110111110010000111110,0,1,0,0,...,0,0,0,0,1,1,1,1,1,0
1,1,00110010000100100100011001,0001100110010000100100100011001,26,0000000000000000000000000010000,0001100110010000100100100001001,0,0,0,1,...,0,1,0,0,0,0,1,0,0,1
2,2,10011010110010011011100100,1011010011010110010011011100100,10,0000000000100000000000000000000,1011010011110110010011011100100,1,0,1,1,...,1,0,1,1,1,0,0,1,0,0
3,3,10001111100101011001110101,1100010001111100101011001110101,21,0000000000000000000001000000000,1100010001111100101010001110101,1,1,0,0,...,0,0,0,1,1,1,0,1,0,1
4,4,01101011110100100000001110,1010001101011110100100000001110,8,0000000010000000000000000000000,1010001111011110100100000001110,1,0,1,0,...,0,0,0,0,0,0,1,1,1,0


In [0]:
train_data, test_data, train_labels, test_labels = \
      model_selection.train_test_split(data.loc[:, 'defective_codeword':'bin_30'], 
                                       data['plainword'], 
                                       test_size = 0.3)

In [0]:
# Create feature columns for all features.
feature_columns = []
for key in train_data.keys():
    feature_columns.append(tf.feature_column.numeric_column(key=key))

In [50]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    hidden_units=[100, 100, 100],
    n_classes=data.shape[0])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp0wue7khv', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9bd7e00fd0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [51]:
classifier.train(
    input_fn=lambda:train_input_fn(train_data, train_labels, args.batch_size),
    steps=args.train_steps)

NameError: ignored